<a href="https://colab.research.google.com/github/mcdelavega/Handling-missing-values-in-histogram-data/blob/master/All_methods_aps_normalized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns  
import pandas.util.testing as tm

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['aps_failure_training_set.csv']), sep=",")

In [ ]:
df.replace(to_replace = 'na', value = np.NaN, inplace = True)
df.replace(to_replace = 'neg', value = 0, inplace = True)
df.replace(to_replace = 'pos', value = 1, inplace = True)

X_all = df.drop(['class', 'aa_000', 'ab_000', 'ac_000', 'ad_000', 'ae_000', 'af_000', 'ah_000', 'ai_000', 'aj_000', 'ak_000', 'al_000', 'am_0', 'an_000', 'ao_000', 'ap_000', 'aq_000', 'ar_000', 'as_000', 'at_000', 'au_000', 'av_000', 'ax_000', 'bb_000', 'bc_000', 'bd_000', 'be_000', 'bf_000', 'bg_000', 'bh_000', 'bi_000', 'bj_000', 'bk_000', 'bl_000', 'bm_000', 'bn_000', 'bo_000', 'bp_000', 'bq_000', 'br_000', 'bs_000', 'bt_000', 'bu_000', 'bv_000', 'bx_000', 'by_000', 'bz_000', 'ca_000', 'cb_000', 'cc_000', 'cd_000', 'ce_000', 'cf_000', 'cg_000', 'ch_000', 'ci_000', 'cj_000', 'ck_000', 'cl_000', 'cm_000', 'co_000', 'cp_000', 'cq_000', 'cr_000', 'ct_000', 'cu_000', 'cv_000', 'cx_000', 'cy_000', 'cz_000', 'da_000', 'db_000', 'dc_000', 'dd_000', 'de_000', 'df_000', 'dg_000', 'dh_000', 'di_000', 'dj_000', 'dk_000', 'dl_000', 'dm_000', 'dn_000', 'do_000', 'dp_000', 'dq_000', 'dr_000', 'ds_000', 'dt_000', 'du_000', 'dv_000', 'dx_000', 'dy_000', 'dz_000', 'ea_000', 'eb_000', 'ec_00', 'ed_000', 'ef_000', 'eg_000'], axis = 1)
y_all = df['class']

train = X_all
cols = train.columns

In [ ]:
#STANDARD IMPUTATIONS
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

#KNN
knn_imputer = KNNImputer(n_neighbors=5)
knn_imputed = knn_imputer.fit_transform(train)

#transpose knn
knn_list = list()
knn_list.append(knn_imputed)

a= {}
cnt=0
for i in knn_list:
    i.transpose()
    i= np.array(i).transpose()
    for j in i:
        a[cols[cnt]]=j
        cnt=cnt+1  
knnDFn = pd.DataFrame(a)

#MEAN
mean_imputer = SimpleImputer(strategy='mean')
mean_imputed = mean_imputer.fit_transform(train)
#transpose mean
mean_list = list()
mean_list.append(mean_imputed)

b= {}
cnt=0
for i in mean_list:
    i.transpose()
    i= np.array(i).transpose()
    for j in i:
        b[cols[cnt]]=j
        cnt=cnt+1  
meanDFn = pd.DataFrame(b)


#MEDIAN
median_imputer = SimpleImputer(strategy='median')
median_imputed = median_imputer.fit_transform(train)

#transpose median
median_list = list()
median_list.append(median_imputed)

c= {}
cnt=0
for i in median_list:
    i.transpose()
    i= np.array(i).transpose()
    for j in i:
        c[cols[cnt]]=j
        cnt=cnt+1  
medianDFn = pd.DataFrame(c)

In [ ]:
#THIS IS THE NORMALIZATION CELL for MEDIAN
#CREATE A LIST OF ALL HISTOGRAMS
median_df = medianDFn
cols = medianDFn.columns
medianlist_hist = list()
mediantemporary_list = list()
for i, j in enumerate(cols[:-1]):

  first_column = j
  next_column = cols[i+1]
  if  i == (len(cols)-2):
    if first_column[0:2] == next_column[0:2]:
      mediantemporary_list.append(median_df[first_column])
      mediantemporary_list.append(median_df[next_column])
      medianlist_hist.append(mediantemporary_list) 
    
    else:
      mediantemporary_list.append(median_df[first_column])
      medianlist_hist.append(mediantemporary_list) 
      #throwing the old temporary away and create a new empty "temporary list"
      mediantemporary_list = list()
      mediantemporary_list.append(median_df[next_column])
      medianlist_hist.append(mediantemporary_list) 

  elif first_column[0:2] == next_column[0:2]:
    mediantemporary_list.append(median_df[first_column])

  else: 
    #add the last column to the temporary list
    mediantemporary_list.append(median_df[first_column])
    medianlist_hist.append(mediantemporary_list) 
    #throwing the old temporary away and create a new empty "temporary list"
    mediantemporary_list = list()


#CREATE NEW LIST AND REPLACE THE ITEMS WITH THEIR WEIGHTS
medianlist_hist2 = list()

for histogram in medianlist_hist:
  sum = 0
  for item in histogram:
    sum += item
  for item in histogram:
    j=item
    for index, itemsItem in enumerate(item):
      if sum[index]==0:
        j[index] = itemsItem
      else:
        j[index] = itemsItem/sum[index]
  
    medianlist_hist2.append(j)

#TRANSPOSE THE LIST TO A DATAFRAME ONCE AGAIN
medianDF = pd.DataFrame(np.transpose(medianlist_hist2),columns=cols)


In [ ]:
#THIS IS THE NORMALIZATION CELL for MEAN
#CREATE A LIST OF ALL HISTOGRAMS
mean_df = meanDFn
cols = meanDFn.columns
meanlist_hist = list()
meantemporary_list = list()
for i, j in enumerate(cols[:-1]):

  first_column = j
  next_column = cols[i+1]
  if  i == (len(cols)-2):
    if first_column[0:2] == next_column[0:2]:
      meantemporary_list.append(mean_df[first_column])
      meantemporary_list.append(mean_df[next_column])
      meanlist_hist.append(meantemporary_list) 
    
    else:
      meantemporary_list.append(mean_df[first_column])
      meanlist_hist.append(meantemporary_list) 
      #throwing the old temporary away and create a new empty "temporary list"
      meantemporary_list = list()
      meantemporary_list.append(mean_df[next_column])
      meanlist_hist.append(meantemporary_list) 

  elif first_column[0:2] == next_column[0:2]:
    meantemporary_list.append(mean_df[first_column])

  else: 
    #add the last column to the temporary list
    meantemporary_list.append(mean_df[first_column])
    meanlist_hist.append(meantemporary_list) 
    #throwing the old temporary away and create a new empty "temporary list"
    meantemporary_list = list()


#CREATE NEW LIST AND REPLACE THE ITEMS WITH THEIR WEIGHTS
meanlist_hist2 = list()

for histogram in meanlist_hist:
  sum = 0
  for item in histogram:
    sum += item
  for item in histogram:
    j=item
    for index, itemsItem in enumerate(item):
      if sum[index]==0:
        j[index] = itemsItem
      else:
        j[index] = itemsItem/sum[index]
  
    meanlist_hist2.append(j)

#TRANSPOSE THE LIST TO A DATAFRAME ONCE AGAIN
meanDF = pd.DataFrame(np.transpose(meanlist_hist2),columns=cols)

In [ ]:
#THIS IS THE NORMALIZATION CELL for KNN
#CREATE A LIST OF ALL HISTOGRAMS
knn_df = knnDFn
cols = knnDFn.columns
knnlist_hist = list()
knntemporary_list = list()
for i, j in enumerate(cols[:-1]):

  first_column = j
  next_column = cols[i+1]
  if  i == (len(cols)-2):
    if first_column[0:2] == next_column[0:2]:
      knntemporary_list.append(knn_df[first_column])
      knntemporary_list.append(knn_df[next_column])
      knnlist_hist.append(knntemporary_list) 
    
    else:
      knntemporary_list.append(knn_df[first_column])
      knnlist_hist.append(knntemporary_list) 
      #throwing the old temporary away and create a new empty "temporary list"
      knntemporary_list = list()
      knntemporary_list.append(knn_df[next_column])
      knnlist_hist.append(knntemporary_list) 

  elif first_column[0:2] == next_column[0:2]:
    knntemporary_list.append(knn_df[first_column])

  else: 
    #add the last column to the temporary list
    knntemporary_list.append(knn_df[first_column])
    knnlist_hist.append(knntemporary_list) 
    #throwing the old temporary away and create a new empty "temporary list"
    knntemporary_list = list()


#CREATE NEW LIST AND REPLACE THE ITEMS WITH THEIR WEIGHTS
knnlist_hist2 = list()

for histogram in knnlist_hist:
  sum = 0
  for item in histogram:
    sum += item
  for item in histogram:
    j=item
    for index, itemsItem in enumerate(item):
      if sum[index]==0:
        j[index] = itemsItem
      else:
        j[index] = itemsItem/sum[index]
  
    knnlist_hist2.append(j)

#TRANSPOSE THE LIST TO A DATAFRAME ONCE AGAIN
knnDF = pd.DataFrame(np.transpose(knnlist_hist2),columns=cols)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

meanDF_scaled = scaler.fit_transform(meanDFn)
medianDF_scaled = scaler.fit_transform(medianDFn)
knnDF_scaled = scaler.fit_transform(knnDFn)

In [ ]:
#KNN_1.0

#PREPROCESS
df = train
cols = df.columns
list_hist = list()
temporary_list = list()
for i, j in enumerate(cols[:-1]):

  first_column = j
  next_column = cols[i+1]
  if  i == (len(cols)-2):
    if first_column[0:2] == next_column[0:2]:
      temporary_list.append(df[first_column])
      temporary_list.append(df[next_column])
      list_hist.append(temporary_list) 
    
    else:
      temporary_list.append(df[first_column])
      list_hist.append(temporary_list) 
      #throwing the old temporary away and create a new empty "temporary list"
      temporary_list = list()
      temporary_list.append(df[next_column])
      list_hist.append(temporary_list) 

  elif first_column[0:2] == next_column[0:2]:
    temporary_list.append(df[first_column])

  else: 
    #add the last column to the temporary list
    temporary_list.append(df[first_column])
    list_hist.append(temporary_list) 
    #throwing the old temporary away and create a new empty "temporary list"
    temporary_list = list()

list_hist2 = list()
temp_list = list()
for histogram in list_hist:
  temp_list= list()
  for column_index, column in enumerate(histogram):    
    for row_index, row in enumerate(column):
      if column_index == 0:
        l = list()
        l.append(row)
        temp_list.append(l)
      else:
        temp_list[row_index].append(row)
    list_hist2.append(temp_list)

list_of_vectors_in_many_histograms = []
for word in list_hist2:
    if word not in list_of_vectors_in_many_histograms:
        list_of_vectors_in_many_histograms.append(word)

list_of_vectors_in_one_list = []
for index, i in enumerate(list_of_vectors_in_many_histograms):
  for index, j in enumerate(i):
    list_of_vectors_in_one_list.append(j)

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
KNN_1 = imputer.fit_transform(list_of_vectors_in_one_list)

cols = X_all.columns
e = pd.DataFrame(KNN_1)
e1 = pd.DataFrame(e.iloc[0:60000])
e1= e1.reset_index(drop=True)
e2 = pd.DataFrame(e.iloc[60000:120000])
e2 = e2.reset_index(drop=True)
e3 = pd.DataFrame(e.iloc[120000:180000])
e3 = e3.reset_index(drop=True)
e4 = pd.DataFrame(e.iloc[180000:240000])
e4 = e4.reset_index(drop=True)
e5 = pd.DataFrame(e.iloc[240000:300000])
e5 = e5.reset_index(drop=True)
e6 = pd.DataFrame(e.iloc[300000:360000])
e6 = e6.reset_index(drop=True)
e7 = pd.DataFrame(e.iloc[360000:420000])
e7 = e7.reset_index(drop=True)

knn_onen = pd.concat([e1,e2, e3, e4, e5, e6, e7], axis=1, sort=False)
knn_onen.columns = [np.arange(0,knn_onen.shape[1])]
knn_onen.columns = ['ag_000', 'ag_001', 'ag_002', 'ag_003', 'ag_004', 'ag_005', 'ag_006',
       'ag_007', 'ag_008', 'ag_009', 'ay_000', 'ay_001', 'ay_002', 'ay_003',
       'ay_004', 'ay_005', 'ay_006', 'ay_007', 'ay_008', 'ay_009', 'az_000',
       'az_001', 'az_002', 'az_003', 'az_004', 'az_005', 'az_006', 'az_007',
       'az_008', 'az_009', 'ba_000', 'ba_001', 'ba_002', 'ba_003', 'ba_004',
       'ba_005', 'ba_006', 'ba_007', 'ba_008', 'ba_009', 'cn_000', 'cn_001',
       'cn_002', 'cn_003', 'cn_004', 'cn_005', 'cn_006', 'cn_007', 'cn_008',
       'cn_009', 'cs_000', 'cs_001', 'cs_002', 'cs_003', 'cs_004', 'cs_005',
       'cs_006', 'cs_007', 'cs_008', 'cs_009', 'ee_000', 'ee_001', 'ee_002',
       'ee_003', 'ee_004', 'ee_005', 'ee_006', 'ee_007', 'ee_008', 'ee_009']

In [ ]:
#THIS IS THE NORMALIZATION CELL for KNN_1.0
#CREATE A LIST OF ALL HISTOGRAMS
knn1_df = knn_onen
cols = knn_onen.columns
knn1list_hist = list()
knn1temporary_list = list()
for i, j in enumerate(cols[:-1]):

  first_column = j
  next_column = cols[i+1]
  if  i == (len(cols)-2):
    if first_column[0:2] == next_column[0:2]:
      knn1temporary_list.append(knn1_df[first_column])
      knn1temporary_list.append(knn1_df[next_column])
      knn1list_hist.append(knn1temporary_list) 
    
    else:
      knn1temporary_list.append(knn1_df[first_column])
      knn1list_hist.append(knn1temporary_list) 
      #throwing the old temporary away and create a new empty "temporary list"
      knn1temporary_list = list()
      knn1temporary_list.append(knn1_df[next_column])
      knn1list_hist.append(knn1temporary_list) 

  elif first_column[0:2] == next_column[0:2]:
    knn1temporary_list.append(knn1_df[first_column])

  else: 
    #add the last column to the temporary list
    knn1temporary_list.append(knn1_df[first_column])
    knn1list_hist.append(knn1temporary_list) 
    #throwing the old temporary away and create a new empty "temporary list"
    knn1temporary_list = list()


#CREATE NEW LIST AND REPLACE THE ITEMS WITH THEIR WEIGHTS
knn1list_hist2 = list()

for histogram in knn1list_hist:
  sum = 0
  for item in histogram:
    sum += item
  for item in histogram:
    j=item
    for index, itemsItem in enumerate(item):
      if sum[index]==0:
        j[index] = itemsItem
      else:
        j[index] = itemsItem/sum[index]
  
    knn1list_hist2.append(j)

#TRANSPOSE THE LIST TO A DATAFRAME ONCE AGAIN
knn_one = pd.DataFrame(np.transpose(knn1list_hist2),columns=cols)

In [ ]:
#KNN_2.0

df1 = X_all.transpose()

temporary_list= list()
final_hist_list= list()
counter=0
for i in range(len(df1.columns)):
    counter=0
    temp=cols[0][:2]
    for j in cols:
        if temp!=j[:2]:
            counter=counter+1
            final_hist_list.append(temporary_list)
            temporary_list= list()
            temp = j[:2]
        temporary_list.append(df1[i][j])
    final_hist_list.append(temporary_list)
    temporary_list=list()
    counter=counter+1

i=0

last_hist_list=list()
for i in range(counter):
    last_hist_list.append([])
i=0    
while i < len(final_hist_list):
    for j in range(counter):
        last_hist_list[j].append(final_hist_list[i])
        i=i+1 
        
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
for i in range(counter):
    last_hist_list[i] = imputer.fit_transform(last_hist_list[i])

u= {}
cnty=0
for i in last_hist_list:
    i.transpose()
    i= np.array(i).transpose()
    for j in i:
        u[cols[cnty]]=j
        cnty=cnty+1    
knn_twon = pd.DataFrame(u)

In [ ]:
#THIS IS THE NORMALIZATION CELL for KNN_2.0
#CREATE A LIST OF ALL HISTOGRAMS
knn2_df = knn_twon
cols = knn_twon.columns
knn2list_hist = list()
knn2temporary_list = list()
for i, j in enumerate(cols[:-1]):

  first_column = j
  next_column = cols[i+1]
  if  i == (len(cols)-2):
    if first_column[0:2] == next_column[0:2]:
      knn2temporary_list.append(knn2_df[first_column])
      knn2temporary_list.append(knn2_df[next_column])
      knn2list_hist.append(knn2temporary_list) 
    
    else:
      knn2temporary_list.append(knn2_df[first_column])
      knn2list_hist.append(knn2temporary_list) 
      #throwing the old temporary away and create a new empty "temporary list"
      knn2temporary_list = list()
      knn2temporary_list.append(knn2_df[next_column])
      knn2list_hist.append(knn2temporary_list) 

  elif first_column[0:2] == next_column[0:2]:
    knn2temporary_list.append(knn2_df[first_column])

  else: 
    #add the last column to the temporary list
    knn2temporary_list.append(knn2_df[first_column])
    knn2list_hist.append(knn2temporary_list) 
    #throwing the old temporary away and create a new empty "temporary list"
    knn2temporary_list = list()


#CREATE NEW LIST AND REPLACE THE ITEMS WITH THEIR WEIGHTS
knn2list_hist2 = list()

for histogram in knn2list_hist:
  sum = 0
  for item in histogram:
    sum += item
  for item in histogram:
    j=item
    for index, itemsItem in enumerate(item):
      if sum[index]==0:
        j[index] = itemsItem
      else:
        j[index] = itemsItem/sum[index]
  
    knn2list_hist2.append(j)

#TRANSPOSE THE LIST TO A DATAFRAME ONCE AGAIN
knn_two = pd.DataFrame(np.transpose(knn2list_hist2),columns=cols)

In [ ]:
#SCALE KNN_1.0 and 2.0 as well.

knn_one_scaled = scaler.fit_transform(knn_onen)
knn_two_scaled = scaler.fit_transform(knn_twon)

In [ ]:
#This is done to enable XGBoost to work
knn = knnDF.values
mean = meanDF.values
median = medianDF.values
knn1 = knn_one.values
knn2 = knn_two.values

In [ ]:
from sklearn.model_selection import train_test_split

#All with 'v' are used only when applying XGBoost, all with 'sc' indicates the scaled dataframes and the rest are applied with hist_norm

Xknn_train, Xknn_test, yknn_train, yknn_test = train_test_split (knnDF, y_all, 
                                                    test_size = 0.2,
                                                    random_state = 10)

Xknnv_train, Xknnv_test, yknnv_train, yknnv_test = train_test_split (knn, y_all, 
                                                    test_size = 0.2,
                                                    random_state = 10)

Xknnsc_train, Xknnsc_test, yknnsc_train, yknnsc_test = train_test_split (knnDF_scaled, y_all, 
                                                    test_size = 0.2,
                                                    random_state = 10)

Xmean_train, Xmean_test, ymean_train, ymean_test = train_test_split (meanDF, y_all, 
                                                    test_size = 0.2,
                                                    random_state = 10)

Xmeanv_train, Xmeanv_test, ymeanv_train, ymeanv_test = train_test_split (mean, y_all, 
                                                    test_size = 0.2,
                                                    random_state = 10)

Xmeansc_train, Xmeansc_test, ymeansc_train, ymeansc_test = train_test_split (meanDF_scaled, y_all, 
                                                    test_size = 0.2,
                                                    random_state = 10)

Xmedian_train, Xmedian_test, ymedian_train, ymedian_test = train_test_split (medianDF, y_all, 
                                                    test_size = 0.2,
                                                    random_state = 10)

Xmedianv_train, Xmedianv_test, ymedianv_train, ymedianv_test = train_test_split (median, y_all, 
                                                    test_size = 0.2,
                                                    random_state = 10)

Xmediansc_train, Xmediansc_test, ymediansc_train, ymediansc_test = train_test_split (medianDF_scaled, y_all, 
                                                    test_size = 0.2,
                                                    random_state = 10)

Xknn1_train, Xknn1_test, yknn1_train, yknn1_test = train_test_split (knn_one, y_all, 
                                                    test_size = 0.2,
                                                    random_state = 10)

Xknn1sc_train, Xknn1sc_test, yknn1sc_train, yknn1sc_test = train_test_split (knn_one_scaled, y_all, 
                                                    test_size = 0.2,
                                                    random_state = 10)

Xknn1v_train, Xknn1v_test, yknn1v_train, yknn1v_test = train_test_split (knn1, y_all, 
                                                    test_size = 0.2,
                                                    random_state = 10)

Xknn2_train, Xknn2_test, yknn2_train, yknn2_test = train_test_split (knn_two, y_all, 
                                                    test_size = 0.2,
                                                    random_state = 10)

Xknn2v_train, Xknn2v_test, yknn2v_train, yknn2v_test = train_test_split (knn2, y_all, 
                                                    test_size = 0.2,
                                                    random_state = 10)

Xknn2sc_train, Xknn2sc_test, yknn2sc_train, yknn2sc_test = train_test_split (knn_two_scaled, y_all, 
                                                    test_size = 0.2,
                                                    random_state = 10)

In [ ]:
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier


# Initialize the models
clf_A = LogisticRegression(solver='sag', random_state = 42, max_iter=1000000)
clf_B = SVC(probability=True, random_state = 912, kernel='rbf')
clf_C = xgb.XGBClassifier()
clf_D = RandomForestClassifier(n_estimators = 128, max_depth=50)

#hist_norm data

clf_A.fit(Xknn_train, yknn_train)
clf_B.fit(Xknn_train, yknn_train)
clf_C.fit(Xknnv_train, yknnv_train)
clf_D.fit(Xknn_train, yknn_train)

clf_A.fit(Xmean_train, ymean_train)
clf_B.fit(Xmean_train, ymean_train)
clf_C.fit(Xmeanv_train, ymeanv_train)
clf_D.fit(Xmean_train, ymean_train)

clf_A.fit(Xmedian_train, ymedian_train)
clf_B.fit(Xmedian_train, ymedian_train)
clf_C.fit(Xmedianv_train, ymedianv_train)
clf_D.fit(Xmedian_train, ymedian_train)

clf_A.fit(Xknn1_train, yknn1_train)
clf_B.fit(Xknn1_train, yknn1_train)
clf_C.fit(Xknn1v_train, yknn1v_train)
clf_D.fit(Xknn1_train, yknn1_train)

clf_A.fit(Xknn2_train, yknn2_train)
clf_B.fit(Xknn2_train, yknn2_train)
clf_C.fit(Xknn2v_train, yknn2v_train)
clf_D.fit(Xknn2_train, yknn2_train)

# scaled data
clf_A.fit(Xknnsc_train, yknnsc_train)
clf_B.fit(Xknnsc_train, yknnsc_train)
clf_C.fit(Xknnsc_train, yknnsc_train)
clf_D.fit(Xknnsc_train, yknnsc_train)

clf_A.fit(Xmeansc_train, ymeansc_train)
clf_B.fit(Xmeansc_train, ymeansc_train)
clf_C.fit(Xmeansc_train, ymeansc_train)
clf_D.fit(Xmeansc_train, ymeansc_train)

clf_A.fit(Xmediansc_train, ymediansc_train)
clf_B.fit(Xmediansc_train, ymediansc_train)
clf_C.fit(Xmediansc_train, ymediansc_train)
clf_D.fit(Xmediansc_train, ymediansc_train)

clf_A.fit(Xknn1sc_train, yknn1sc_train)
clf_B.fit(Xknn1sc_train, yknn1sc_train)
clf_C.fit(Xknn1sc_train, yknn1sc_train)
clf_D.fit(Xknn1sc_train, yknn1sc_train)

clf_A.fit(Xknn2sc_train, yknn2sc_train)
clf_B.fit(Xknn2sc_train, yknn2sc_train)
clf_C.fit(Xknn2sc_train, yknn2sc_train)
clf_D.fit(Xknn2sc_train, yknn2sc_train)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score


#********KNN_histnorm******************

#AUC
knn_lr_probs = clf_A.predict_proba(Xknn_test)
knn_svm_probs = clf_B.predict_proba(Xknn_test)
knn_xgb_probs = clf_C.predict_proba(Xknnv_test)
knn_rf_probs = clf_D.predict_proba(Xknn_test)

knn_lr_probs = knn_lr_probs[:, 1]
knn_svm_probs = knn_svm_probs[:, 1]
knn_xgb_probs = knn_xgb_probs[:, 1]
knn_rf_probs = knn_rf_probs[:, 1]
# calculate scores
knn_lr_auc = roc_auc_score(yknn_test, knn_lr_probs)
knn_svm_auc = roc_auc_score(yknn_test, knn_svm_probs)
knn_xgb_auc = roc_auc_score(yknnv_test, knn_xgb_probs)
knn_rf_auc = roc_auc_score(yknn_test, knn_rf_probs)

# OTHER METRICS for standard KNN
knn_lr_pred = clf_A.predict(Xknn_test)
knn_s_pred = clf_B.predict(Xknn_test)
knn_x_pred = clf_C.predict(Xknnv_test)
knn_rf_pred = clf_D.predict(Xknn_test)


#********KNN_scaled******************

#AUC
knnsc_lr_probs = clf_A.predict_proba(Xknnsc_test)
knnsc_svm_probs = clf_B.predict_proba(Xknnsc_test)
knnsc_xgb_probs = clf_C.predict_proba(Xknnsc_test)
knnsc_rf_probs = clf_D.predict_proba(Xknnsc_test)

knnsc_lr_probs = knnsc_lr_probs[:, 1]
knnsc_svm_probs = knnsc_svm_probs[:, 1]
knnsc_xgb_probs = knnsc_xgb_probs[:, 1]
knnsc_rf_probs = knnsc_rf_probs[:, 1]
# calculate scores
knnsc_lr_auc = roc_auc_score(yknnsc_test, knnsc_lr_probs)
knnsc_svm_auc = roc_auc_score(yknnsc_test, knnsc_svm_probs)
knnsc_xgb_auc = roc_auc_score(yknnsc_test, knnsc_xgb_probs)
knnsc_rf_auc = roc_auc_score(yknnsc_test, knnsc_rf_probs)

# OTHER METRICS for standard KNN
knnsc_lr_pred = clf_A.predict(Xknnsc_test)
knnsc_s_pred = clf_B.predict(Xknnsc_test)
knnsc_x_pred = clf_C.predict(Xknnsc_test)
knnsc_rf_pred = clf_D.predict(Xknnsc_test)




#********MEAN_histnorm******************

mean_lr_probs = clf_A.predict_proba(Xmean_test)
mean_svm_probs = clf_B.predict_proba(Xmean_test)
mean_xgb_probs = clf_C.predict_proba(Xmeanv_test)
mean_rf_probs = clf_D.predict_proba(Xmean_test)

mean_lr_probs = mean_lr_probs[:, 1]
mean_svm_probs = mean_svm_probs[:, 1]
mean_xgb_probs = mean_xgb_probs[:, 1]
mean_rf_probs = mean_rf_probs[:, 1]
# calculate scores
mean_lr_auc = roc_auc_score(ymean_test, mean_lr_probs)
mean_svm_auc = roc_auc_score(ymean_test, mean_svm_probs)
mean_xgb_auc = roc_auc_score(ymeanv_test, mean_xgb_probs)
mean_rf_auc = roc_auc_score(ymean_test, mean_rf_probs)

# OTHER METRICS for standard MEAN
mean_lr_pred = clf_A.predict(Xmean_test)
mean_s_pred = clf_B.predict(Xmean_test)
mean_x_pred = clf_C.predict(Xmeanv_test)
mean_rf_pred = clf_D.predict(Xmean_test)


#********MEAN_scaled******************

meansc_lr_probs = clf_A.predict_proba(Xmeansc_test)
meansc_svm_probs = clf_B.predict_proba(Xmeansc_test)
meansc_xgb_probs = clf_C.predict_proba(Xmeansc_test)
meansc_rf_probs = clf_D.predict_proba(Xmeansc_test)

meansc_lr_probs = meansc_lr_probs[:, 1]
meansc_svm_probs = meansc_svm_probs[:, 1]
meansc_xgb_probs = meansc_xgb_probs[:, 1]
meansc_rf_probs = meansc_rf_probs[:, 1]
# calculate scores
meansc_lr_auc = roc_auc_score(ymeansc_test, meansc_lr_probs)
meansc_svm_auc = roc_auc_score(ymeansc_test, meansc_svm_probs)
meansc_xgb_auc = roc_auc_score(ymeansc_test, meansc_xgb_probs)
meansc_rf_auc = roc_auc_score(ymeansc_test, meansc_rf_probs)

# OTHER METRICS for standard MEAN
meansc_lr_pred = clf_A.predict(Xmeansc_test)
meansc_s_pred = clf_B.predict(Xmeansc_test)
meansc_x_pred = clf_C.predict(Xmeansc_test)
meansc_rf_pred = clf_D.predict(Xmeansc_test)



#********MEDIAN_histnorm******************

median_lr_probs = clf_A.predict_proba(Xmedian_test)
median_svm_probs = clf_B.predict_proba(Xmedian_test)
median_xgb_probs = clf_C.predict_proba(Xmedianv_test)
median_rf_probs = clf_D.predict_proba(Xmedian_test)

median_lr_probs = median_lr_probs[:, 1]
median_svm_probs = median_svm_probs[:, 1]
median_xgb_probs = median_xgb_probs[:, 1]
median_rf_probs = median_rf_probs[:, 1]
# calculate scores
median_lr_auc = roc_auc_score(ymedian_test, median_lr_probs)
median_svm_auc = roc_auc_score(ymedian_test, median_svm_probs)
median_xgb_auc = roc_auc_score(ymedianv_test, median_xgb_probs)
median_rf_auc = roc_auc_score(ymedian_test, median_rf_probs)

# OTHER METRICS for standard MEDIAN
median_lr_pred = clf_A.predict(Xmedian_test)
median_s_pred = clf_B.predict(Xmedian_test)
median_x_pred = clf_C.predict(Xmedianv_test)
median_rf_pred = clf_D.predict(Xmedian_test)


#********MEDIAN_scaled******************

mediansc_lr_probs = clf_A.predict_proba(Xmediansc_test)
mediansc_svm_probs = clf_B.predict_proba(Xmediansc_test)
mediansc_xgb_probs = clf_C.predict_proba(Xmediansc_test)
mediansc_rf_probs = clf_D.predict_proba(Xmediansc_test)

mediansc_lr_probs = mediansc_lr_probs[:, 1]
mediansc_svm_probs = mediansc_svm_probs[:, 1]
mediansc_xgb_probs = mediansc_xgb_probs[:, 1]
mediansc_rf_probs = mediansc_rf_probs[:, 1]
# calculate scores
mediansc_lr_auc = roc_auc_score(ymediansc_test, mediansc_lr_probs)
mediansc_svm_auc = roc_auc_score(ymediansc_test, mediansc_svm_probs)
mediansc_xgb_auc = roc_auc_score(ymediansc_test, mediansc_xgb_probs)
mediansc_rf_auc = roc_auc_score(ymediansc_test, mediansc_rf_probs)

# OTHER METRICS for standard MEDIAN
mediansc_lr_pred = clf_A.predict(Xmediansc_test)
mediansc_s_pred = clf_B.predict(Xmediansc_test)
mediansc_x_pred = clf_C.predict(Xmediansc_test)
mediansc_rf_pred = clf_D.predict(Xmediansc_test)



#********KNN_1.0_histnorm******************

#AUC
knn1_lr_probs = clf_A.predict_proba(Xknn1_test)
knn1_svm_probs = clf_B.predict_proba(Xknn1_test)
knn1_xgb_probs = clf_C.predict_proba(Xknn1v_test)
knn1_rf_probs = clf_D.predict_proba(Xknn1_test)

knn1_lr_probs = knn1_lr_probs[:, 1]
knn1_svm_probs = knn1_svm_probs[:, 1]
knn1_xgb_probs = knn1_xgb_probs[:, 1]
knn1_rf_probs = knn1_rf_probs[:, 1]
# calculate scores
knn1_lr_auc = roc_auc_score(yknn1_test, knn1_lr_probs)
knn1_svm_auc = roc_auc_score(yknn1_test, knn1_svm_probs)
knn1_xgb_auc = roc_auc_score(yknn1v_test, knn1_xgb_probs)
knn1_rf_auc = roc_auc_score(yknn1_test, knn1_rf_probs)

# OTHER METRICS for standard KNN_1.0
knn1_lr_pred = clf_A.predict(Xknn1_test)
knn1_s_pred = clf_B.predict(Xknn1_test)
knn1_x_pred = clf_C.predict(Xknn1v_test)
knn1_rf_pred = clf_D.predict(Xknn1_test)


#********KNN_1.0_scaled******************

#AUC
knn1sc_lr_probs = clf_A.predict_proba(Xknn1sc_test)
knn1sc_svm_probs = clf_B.predict_proba(Xknn1sc_test)
knn1sc_xgb_probs = clf_C.predict_proba(Xknn1sc_test)
knn1sc_rf_probs = clf_D.predict_proba(Xknn1sc_test)

knn1sc_lr_probs = knn1sc_lr_probs[:, 1]
knn1sc_svm_probs = knn1sc_svm_probs[:, 1]
knn1sc_xgb_probs = knn1sc_xgb_probs[:, 1]
knn1sc_rf_probs = knn1sc_rf_probs[:, 1]
# calculate scores
knn1sc_lr_auc = roc_auc_score(yknn1sc_test, knn1sc_lr_probs)
knn1sc_svm_auc = roc_auc_score(yknn1sc_test, knn1sc_svm_probs)
knn1sc_xgb_auc = roc_auc_score(yknn1sc_test, knn1sc_xgb_probs)
knn1sc_rf_auc = roc_auc_score(yknn1sc_test, knn1sc_rf_probs)

# OTHER METRICS for standard KNN_1.0
knn1sc_lr_pred = clf_A.predict(Xknn1sc_test)
knn1sc_s_pred = clf_B.predict(Xknn1sc_test)
knn1sc_x_pred = clf_C.predict(Xknn1sc_test)
knn1sc_rf_pred = clf_D.predict(Xknn1sc_test)


#********KNN_2.0_histnorm******************

#AUC
knn2_lr_probs = clf_A.predict_proba(Xknn2_test)
knn2_svm_probs = clf_B.predict_proba(Xknn2_test)
knn2_xgb_probs = clf_C.predict_proba(Xknn2v_test)
knn2_rf_probs = clf_D.predict_proba(Xknn2_test)

knn2_lr_probs = knn2_lr_probs[:, 1]
knn2_svm_probs = knn2_svm_probs[:, 1]
knn2_xgb_probs = knn2_xgb_probs[:, 1]
knn2_rf_probs = knn2_rf_probs[:, 1]
# calculate scores
knn2_lr_auc = roc_auc_score(yknn2_test, knn2_lr_probs)
knn2_svm_auc = roc_auc_score(yknn2_test, knn2_svm_probs)
knn2_xgb_auc = roc_auc_score(yknn2v_test, knn2_xgb_probs)
knn2_rf_auc = roc_auc_score(yknn2_test, knn2_rf_probs)

# OTHER METRICS for standard KNN
knn2_lr_pred = clf_A.predict(Xknn2_test)
knn2_s_pred = clf_B.predict(Xknn2_test)
knn2_x_pred = clf_C.predict(Xknn2v_test)
knn2_rf_pred = clf_D.predict(Xknn2_test)


#********KNN_2.0_scaled******************

#AUC
knn2sc_lr_probs = clf_A.predict_proba(Xknn2sc_test)
knn2sc_svm_probs = clf_B.predict_proba(Xknn2sc_test)
knn2sc_xgb_probs = clf_C.predict_proba(Xknn2sc_test)
knn2sc_rf_probs = clf_D.predict_proba(Xknn2sc_test)

knn2sc_lr_probs = knn2sc_lr_probs[:, 1]
knn2sc_svm_probs = knn2sc_svm_probs[:, 1]
knn2sc_xgb_probs = knn2sc_xgb_probs[:, 1]
knn2sc_rf_probs = knn2sc_rf_probs[:, 1]
# calculate scores
knn2sc_lr_auc = roc_auc_score(yknn2sc_test, knn2sc_lr_probs)
knn2sc_svm_auc = roc_auc_score(yknn2sc_test, knn2sc_svm_probs)
knn2sc_xgb_auc = roc_auc_score(yknn2sc_test, knn2sc_xgb_probs)
knn2sc_rf_auc = roc_auc_score(yknn2sc_test, knn2sc_rf_probs)

# OTHER METRICS for standard KNN
knn2sc_lr_pred = clf_A.predict(Xknn2sc_test)
knn2sc_s_pred = clf_B.predict(Xknn2sc_test)
knn2sc_x_pred = clf_C.predict(Xknn2sc_test)
knn2sc_rf_pred = clf_D.predict(Xknn2sc_test)

#Summarize the scores
print ('KNN with hist_norm')
print ('LOGISTIC REGRESSION')
print('AUC score: %.3f' % (knn_lr_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknn_test, knn_lr_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknn_test, knn_lr_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknn_test, knn_lr_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknn_test, knn_lr_pred)))
print('')
print ('SVM')
print('AUC score: %.3f' % (knn_svm_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknn_test, knn_s_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknn_test, knn_s_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknn_test, knn_s_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknn_test, knn_s_pred)))
print('')
print ('XGBOOST')
print('AUC score: %.3f' % (knn_xgb_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknnv_test, knn_x_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknnv_test, knn_x_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknnv_test, knn_x_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknnv_test, knn_x_pred)))
print('')
print ('RANDOM FOREST')
print('AUC score: %.3f' % (knn_rf_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknn_test, knn_rf_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknn_test, knn_rf_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknn_test, knn_rf_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknn_test, knn_rf_pred)))
print('')
print('')
print('')
print ('MEAN with hist_norm')
print ('LOGISTIC REGRESSION')
print('AUC score: %.3f' % (mean_lr_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(ymean_test, mean_lr_pred)))
print('Recall Score: {:.2f}'.format(recall_score(ymean_test, mean_lr_pred)))
print('Precision Score: {:.2f}'.format(precision_score(ymean_test, mean_lr_pred)))
print('F1 Score: {:.2f}'.format(f1_score(ymean_test, mean_lr_pred)))
print('')
print ('SVM')
print('AUC score: %.3f' % (mean_svm_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(ymean_test, mean_s_pred)))
print('Recall Score: {:.2f}'.format(recall_score(ymean_test, mean_s_pred)))
print('Precision Score: {:.2f}'.format(precision_score(ymean_test, mean_s_pred)))
print('F1 Score: {:.2f}'.format(f1_score(ymean_test, mean_s_pred)))
print('')
print ('XGBOOST')
print('AUC score: %.3f' % (mean_xgb_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(ymeanv_test, mean_x_pred)))
print('Recall Score: {:.2f}'.format(recall_score(ymeanv_test, mean_x_pred)))
print('Precision Score: {:.2f}'.format(precision_score(ymeanv_test, mean_x_pred)))
print('F1 Score: {:.2f}'.format(f1_score(ymeanv_test, mean_x_pred)))
print('')
print ('RANDOM FOREST')
print('AUC score: %.3f' % (mean_rf_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(ymean_test, mean_rf_pred)))
print('Recall Score: {:.2f}'.format(recall_score(ymean_test, mean_rf_pred)))
print('Precision Score: {:.2f}'.format(precision_score(ymean_test, mean_rf_pred)))
print('F1 Score: {:.2f}'.format(f1_score(ymean_test, mean_rf_pred)))
print('')
print('')
print('')
print ('MEDIAN with hist_norm')
print ('LOGISTIC REGRESSION')
print('AUC score: %.3f' % (median_lr_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(ymedian_test, median_lr_pred)))
print('Recall Score: {:.2f}'.format(recall_score(ymedian_test, median_lr_pred)))
print('Precision Score: {:.2f}'.format(precision_score(ymedian_test, median_lr_pred)))
print('F1 Score: {:.2f}'.format(f1_score(ymedian_test, median_lr_pred)))
print('')
print ('SVM')
print('AUC score: %.3f' % (median_svm_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(ymedian_test, median_s_pred)))
print('Recall Score: {:.2f}'.format(recall_score(ymedian_test, median_s_pred)))
print('Precision Score: {:.2f}'.format(precision_score(ymedian_test, median_s_pred)))
print('F1 Score: {:.2f}'.format(f1_score(ymedian_test, median_s_pred)))
print('')
print ('XGBOOST')
print('AUC score: %.3f' % (median_xgb_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(ymedianv_test, median_x_pred)))
print('Recall Score: {:.2f}'.format(recall_score(ymedianv_test, median_x_pred)))
print('Precision Score: {:.2f}'.format(precision_score(ymedianv_test, median_x_pred)))
print('F1 Score: {:.2f}'.format(f1_score(ymedian_test, median_x_pred)))
print('')
print ('RANDOM FOREST')
print('AUC score: %.3f' % (median_rf_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(ymedian_test, median_rf_pred)))
print('Recall Score: {:.2f}'.format(recall_score(ymedian_test, median_rf_pred)))
print('Precision Score: {:.2f}'.format(precision_score(ymedian_test, median_rf_pred)))
print('F1 Score: {:.2f}'.format(f1_score(ymedian_test, median_rf_pred)))
print('')
print('')
print('')
print ('KNN_1.0 with hist_norm')
print ('LOGISTIC REGRESSION')
print('AUC score: %.3f' % (knn1_lr_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknn1_test, knn1_lr_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknn1_test, knn1_lr_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknn1_test, knn1_lr_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknn1_test, knn1_lr_pred)))
print('')
print ('SVM')
print('AUC score: %.3f' % (knn1_svm_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknn1_test, knn1_s_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknn1_test, knn1_s_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknn1_test, knn1_s_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknn1_test, knn1_s_pred)))
print('')
print ('XGBOOST')
print('AUC score: %.3f' % (knn1_xgb_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknn1v_test, knn1_x_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknn1v_test, knn1_x_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknn1v_test, knn1_x_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknn1v_test, knn1_x_pred)))
print('')
print ('RANDOM FOREST')
print('AUC score: %.3f' % (knn1_rf_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknn1_test, knn1_rf_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknn1_test, knn1_rf_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknn1_test, knn1_rf_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknn1_test, knn1_rf_pred)))
print('')
print('')
print('')
print ('KNN_2.0 with hist_norm')
print ('LOGISTIC REGRESSION')
print('AUC score: %.3f' % (knn2_lr_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknn2_test, knn2_lr_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknn2_test, knn2_lr_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknn2_test, knn2_lr_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknn2_test, knn2_lr_pred)))
print('')
print ('SVM')
print('AUC score: %.3f' % (knn2_svm_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknn2_test, knn2_s_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknn2_test, knn2_s_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknn2_test, knn2_s_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknn2_test, knn2_s_pred)))
print('')
print ('XGBOOST')
print('AUC score: %.3f' % (knn2_xgb_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknn2v_test, knn2_x_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknn2v_test, knn2_x_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknn2v_test, knn2_x_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknn2v_test, knn2_x_pred)))
print('')
print ('RANDOM FOREST')
print('AUC score: %.3f' % (knn2_rf_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknn2_test, knn2_rf_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknn2_test, knn2_rf_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknn2_test, knn2_rf_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknn2_test, knn2_rf_pred)))
print('')
print('')
print('')
print ('KNN scaled')
print ('LOGISTIC REGRESSION')
print('AUC score: %.3f' % (knnsc_lr_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknnsc_test, knnsc_lr_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknnsc_test, knnsc_lr_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknnsc_test, knnsc_lr_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknnsc_test, knnsc_lr_pred)))
print('')
print ('SVM')
print('AUC score: %.3f' % (knnsc_svm_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknnsc_test, knnsc_s_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknnsc_test, knnsc_s_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknnsc_test, knnsc_s_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknnsc_test, knnsc_s_pred)))
print('')
print ('XGBOOST')
print('AUC score: %.3f' % (knnsc_xgb_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknnsc_test, knnsc_x_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknnsc_test, knnsc_x_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknnsc_test, knnsc_x_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknnsc_test, knnsc_x_pred)))
print('')
print ('RANDOM FOREST')
print('AUC score: %.3f' % (knnsc_rf_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknnsc_test, knnsc_rf_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknnsc_test, knnsc_rf_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknnsc_test, knnsc_rf_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknnsc_test, knnsc_rf_pred)))
print('')
print('')
print('')
print ('MEAN scaled')
print ('LOGISTIC REGRESSION')
print('AUC score: %.3f' % (meansc_lr_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(ymeansc_test, meansc_lr_pred)))
print('Recall Score: {:.2f}'.format(recall_score(ymeansc_test, meansc_lr_pred)))
print('Precision Score: {:.2f}'.format(precision_score(ymeansc_test, meansc_lr_pred)))
print('F1 Score: {:.2f}'.format(f1_score(ymeansc_test, meansc_lr_pred)))
print('')
print ('SVM')
print('AUC score: %.3f' % (meansc_svm_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(ymeansc_test, meansc_s_pred)))
print('Recall Score: {:.2f}'.format(recall_score(ymeansc_test, meansc_s_pred)))
print('Precision Score: {:.2f}'.format(precision_score(ymeansc_test, meansc_s_pred)))
print('F1 Score: {:.2f}'.format(f1_score(ymeansc_test, meansc_s_pred)))
print('')
print ('XGBOOST')
print('AUC score: %.3f' % (meansc_xgb_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(ymeansc_test, meansc_x_pred)))
print('Recall Score: {:.2f}'.format(recall_score(ymeansc_test, meansc_x_pred)))
print('Precision Score: {:.2f}'.format(precision_score(ymeansc_test, meansc_x_pred)))
print('F1 Score: {:.2f}'.format(f1_score(ymeansc_test, meansc_x_pred)))
print('')
print ('RANDOM FOREST')
print('AUC score: %.3f' % (meansc_rf_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(ymeansc_test, meansc_rf_pred)))
print('Recall Score: {:.2f}'.format(recall_score(ymeansc_test, meansc_rf_pred)))
print('Precision Score: {:.2f}'.format(precision_score(ymeansc_test, meansc_rf_pred)))
print('F1 Score: {:.2f}'.format(f1_score(ymeansc_test, meansc_rf_pred)))
print('')
print('')
print('')
print ('MEDIAN scaled')
print ('LOGISTIC REGRESSION')
print('AUC score: %.3f' % (mediansc_lr_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(ymediansc_test, mediansc_lr_pred)))
print('Recall Score: {:.2f}'.format(recall_score(ymediansc_test, mediansc_lr_pred)))
print('Precision Score: {:.2f}'.format(precision_score(ymediansc_test, mediansc_lr_pred)))
print('F1 Score: {:.2f}'.format(f1_score(ymediansc_test, mediansc_lr_pred)))
print('')
print ('SVM')
print('AUC score: %.3f' % (mediansc_svm_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(ymediansc_test, mediansc_s_pred)))
print('Recall Score: {:.2f}'.format(recall_score(ymediansc_test, mediansc_s_pred)))
print('Precision Score: {:.2f}'.format(precision_score(ymediansc_test, mediansc_s_pred)))
print('F1 Score: {:.2f}'.format(f1_score(ymediansc_test, mediansc_s_pred)))
print('')
print ('XGBOOST')
print('AUC score: %.3f' % (mediansc_xgb_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(ymediansc_test, mediansc_x_pred)))
print('Recall Score: {:.2f}'.format(recall_score(ymediansc_test, mediansc_x_pred)))
print('Precision Score: {:.2f}'.format(precision_score(ymediansc_test, mediansc_x_pred)))
print('F1 Score: {:.2f}'.format(f1_score(ymediansc_test, mediansc_x_pred)))
print('')
print ('RANDOM FOREST')
print('AUC score: %.3f' % (mediansc_rf_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(ymediansc_test, mediansc_rf_pred)))
print('Recall Score: {:.2f}'.format(recall_score(ymediansc_test, mediansc_rf_pred)))
print('Precision Score: {:.2f}'.format(precision_score(ymediansc_test, mediansc_rf_pred)))
print('F1 Score: {:.2f}'.format(f1_score(ymediansc_test, mediansc_rf_pred)))
print('')
print('')
print('')
print ('KNN_1.0 scaled')
print ('LOGISTIC REGRESSION')
print('AUC score: %.3f' % (knn1sc_lr_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknn1sc_test, knn1sc_lr_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknn1sc_test, knn1sc_lr_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknn1sc_test, knn1sc_lr_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknn1sc_test, knn1sc_lr_pred)))
print('')
print ('SVM')
print('AUC score: %.3f' % (knn1sc_svm_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknn1sc_test, knn1sc_s_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknn1sc_test, knn1sc_s_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknn1sc_test, knn1sc_s_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknn1sc_test, knn1sc_s_pred)))
print('')
print ('XGBOOST')
print('AUC score: %.3f' % (knn1sc_xgb_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknn1sc_test, knn1sc_x_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknn1sc_test, knn1sc_x_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknn1sc_test, knn1sc_x_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknn1sc_test, knn1sc_x_pred)))
print('')
print ('RANDOM FOREST')
print('AUC score: %.3f' % (knn1sc_rf_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknn1sc_test, knn1sc_rf_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknn1sc_test, knn1sc_rf_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknn1sc_test, knn1sc_rf_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknn1sc_test, knn1sc_rf_pred)))
print('')
print('')
print('')
print ('KNN_2.0 scaled')
print ('LOGISTIC REGRESSION')
print('AUC score: %.3f' % (knn2sc_lr_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknn2sc_test, knn2sc_lr_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknn2sc_test, knn2sc_lr_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknn2sc_test, knn2sc_lr_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknn2sc_test, knn2sc_lr_pred)))
print('')
print ('SVM')
print('AUC score: %.3f' % (knn2sc_svm_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknn2sc_test, knn2sc_s_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknn2sc_test, knn2sc_s_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknn2sc_test, knn2sc_s_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknn2sc_test, knn2sc_s_pred)))
print('')
print ('XGBOOST')
print('AUC score: %.3f' % (knn2sc_xgb_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknn2sc_test, knn2sc_x_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknn2sc_test, knn2sc_x_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknn2sc_test, knn2sc_x_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknn2sc_test, knn2sc_x_pred)))
print('')
print ('RANDOM FOREST')
print('AUC score: %.3f' % (knn2sc_rf_auc))
print('Accuracy Score: {:.2f}'.format(accuracy_score(yknn2sc_test, knn2sc_rf_pred)))
print('Recall Score: {:.2f}'.format(recall_score(yknn2sc_test, knn2sc_rf_pred)))
print('Precision Score: {:.2f}'.format(precision_score(yknn2sc_test, knn2sc_rf_pred)))
print('F1 Score: {:.2f}'.format(f1_score(yknn2sc_test, knn2sc_rf_pred)))
